Глава 7. Очистка и подготовка данных

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

- Значительная часть времени программиста, занимающегося анализом и моделированием данных, уходит на подготовку данных: загрузку, очистку, преобразование и реорганизацию. Часто говорят, что это составляет 80 % и даже
более времени работы аналитика.
- В этой главе мы обсудим средства работы с отсутствующими и повторяющимися данными, средства обработки строк и некоторые другие преобразования данных, применяемые в процессе анализа.

# 7.1. Обработка отсутствующих данных

В pandas для представления отсутствующих данных с плавающей точкой используется значение NaN
(не число). Это просто признак, который легко распознать:

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
# или None
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
# Встроенное в Python значение None также рассматривается как
# отсутствующее в массивах объектов:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

Таблица 7.1. Методы обработки отсутствующих данных

- dropna - Фильтрует метки оси в зависимости от того, существуют ли для метки отсутствующие данные,
причем есть возможность указать различные пороги, определяющие, какое количество
отсутствующих данных считать допустимым
- fillna - Восполняет отсутствующие данные указанным значением или использует какой-нибудь
метод интерполяции, например 'ffill' или 'bfill'
- isnull - Возвращает объект, содержащий булевы значения, которые показывают, какие значения
отсутствуют
- notnull - Логическое отрицание isnull

## Фильтрация отсутствующих данных

In [5]:
# Метод dropna для Series возвращает другой объект Series, содержащий
# только данные и значения индекса, отличные от NA (В pandas принято
# соглашение обозначать отсутствующие данные NA - Not Available):

from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [6]:
# Эквивалентно: 

data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

В случае объектов DataFrame можно отбрасывать
строки или столбцы, если они содержат только NA-значения или хотя бы
одно NA-значение.
- По умолчанию метод dropna отбрасывает все строки, содержащие хотя бы одно отсутствующее значение:

In [7]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
# Для столбцов axis=1
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [8]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [9]:
# Если передать параметр how='all', то будут отброшены строки, которые
# целиком состоят из отсутствующих значений:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Родственный способ фильтрации строк DataFrame в основном применяется
к временным рядам. Допустим, требуется оставить только строки, содержащие определенное количество наблюдений. Этот порог можно задать с помощью аргумента thresh:

In [10]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [11]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [12]:
# Выводим строки, в которых есть 2 и более не нулевых значения:
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


## Восполнение отсутствующих данных

Иногда отсутствующие данные нужно каким-то способом
заполнить. В большинстве случаев для этой цели можно использовать
метод fillna. Ему передается константа, подставляемая вместо отсутствующих значений:

In [13]:
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [14]:
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [15]:
# Если передать методу fillna словарь, то можно будет подставлять вместо
# отсутствующих данных значение, зависящее от столбца:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [16]:
# Метод fillna возвращает новый объект, но можно также модифицировать
# существующий объект на месте:
df.fillna(0, inplace=True)
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


Те же методы интерполяции, что применяются для переиндексации, годятся и для fillna:

In [17]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


In [18]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


Таблица 7.2. Аргументы метода fillna

- value - Скалярное значение или похожий на словарь объект для восполнения отсутствующих
значений
- method - Метод интерполяции. По умолчанию, если не задано других аргументов, предполагается
метод 'ffill'
- axis - Ось, по которой производится восполнение. По умолчанию axis=0
- inplace - Модифицировать исходный объект, не создавая копию
- limit - Для прямого и обратного восполнения максимальное количество непрерывных
заполняемых промежутков

# 7.2. Преобразование данных

## Устранение дубликатов

In [19]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 5, 
                  'k2': [1, 1, 2, 3, 3, 4, 4, 4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4
7,two,4


In [20]:
# Метод duplicated объекта DataFrame возвращает булев объект Series,
# который для каждой строки показывает, есть в ней дубликаты или нет:

data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
7     True
dtype: bool

In [21]:
# метод drop_duplicates возвращает DataFrame без дублированных строк

data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


По умолчанию оба метода принимают во внимание все столбцы, но можно
указать произвольное подмножество столбцов, которые необходимо исследовать на наличие дубликатов.

In [22]:
# Допустим, есть еще один столбец значений, и мы хотим отфильтровать
# строки, которые содержат повторяющиеся значения в столбце 'k1':

data['v1'] = range(8)

data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6
7,two,4,7


In [23]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


По умолчанию методы duplicated и drop_duplicates оставляют первую встретившуюся строку с данной комбинацией значений. Но если задать параметр
keep='last', то будет оставлена последняя строка:

In [24]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
7,two,4,7


## Преобразование данных с помощью функции или отображения

In [25]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [26]:
# Допустим, требуется добавить столбец, в котором указано соответствующее
# сорту мяса животное. Создадим отображение сортов мяса на виды животных:

meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [27]:
# Создадим объект Series, в котором все названия будут в нижнем регистре:

lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [28]:
# Теперь можно воспользоваться методом map, чтобы сделать сопоставленный столбец
# После того как мы сделали сопоставленный столбец мы добавляем его в таблицу

# Метод map – удобное средство выполнения поэлементных преобразований
# и других операций очистки.

data['animal'] = lowercased.map(meat_to_animal)

data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [29]:
# Можно было бы также передать функцию, выполняющую всю эту работу:

data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

## Замена значений

In [30]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

Значение –999 могло бы быть маркером отсутствующих данных. Чтобы заменить все такие значения теми, которые понимает pandas, воспользуемся
методом replace, порождающим новый объект Series (если только не передан
аргумент inplace=True):

In [31]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [32]:
# Чтобы заменить сразу несколько значений, нужно передать их список
# и заменяющее значение:

data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [33]:
# Если для каждого заменяемого значения нужно свое заменяющее, 
# передаем список замен:

data.replace([-999, -1000], [np.nan, 0])
# или так: data.replace({–999: np.nan, –1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

P.S. Метод data.replace не то же самое, что метод data.str.replace, который выполняет поэлементную замену строки.

## Переименование индексов осей

In [34]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [35]:
# Как и у объекта Series, у индексов осей имеется метод map:

transform = lambda x: x[:4].upper()
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [36]:
# Если требуется создать преобразованный вариант набора данных, не меняя
# оригинал, то будет полезен метод rename:

# Делаем назв. строк маленькими, а назв. столбцов заглавными:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [37]:
# rename можно использовать в сочетании с похожим на словарь объектом,
# который возвращает новые значения для подмножества меток оси:

data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## Дискретизация и раскладывание

Имеются
данные о группе лиц в каком-то исследовании, и требуется разложить их по
ящикам, соответствующим возрасту – дискретной величине:

In [38]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Разобьем эти ящики на группы: от 18 до 25, от 26 до 35, от 35 до 60 и наконец от 61. Для этой цели в pandas есть функция cut:

In [39]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

Pandas возвращает специальный объект Categorical. В атрибуте codes метки данных:

In [40]:
# т.е. (18, 25] == 0, (25, 35] == 1 и т.д.

cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [41]:
# посчитаем количество:

pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

Согласно принятой в математике нотации интервалов, круглая скобка
означает, что соответствующий конец не включается (открыт), а квадратная – что включается (замкнут). Чтобы сделать открытым правый конец,
следует задать параметр right=False:

In [42]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

Можно также самостоятельно задать имена ящиков, передав список или
массив в параметре labels:

In [43]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [44]:
catss = pd.cut(ages, bins, labels=group_names)
pd.value_counts(catss)

Youth         5
YoungAdult    3
MiddleAged    3
Senior        1
dtype: int64

In [45]:
###

## Обнаружение и фильтрация выбросов

Фильтрация или преобразование выбросов – это в основном вопрос применения операций с массивами. 

In [46]:
# Рассмотрим объект DataFrame с нормально распределенными данными:

data = pd.DataFrame(np.random.randn(1000, 4))

data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.000987,-0.060614,0.070131,0.019100
std,0.996438,0.995017,0.992031,1.002271
min,-3.745356,-3.428254,-3.548824,-3.184377
25%,-0.636641,-0.771156,-0.591841,-0.641675
50%,-0.010997,-0.115171,0.094503,-0.000015
75%,0.659019,0.624615,0.787953,0.676536
max,3.927528,3.366626,2.653656,3.260383


In [47]:
# Допустим, мы хотим найти в одном из столбцов значения, превышающие
# 3 по абсолютной величине:

col = data[2]

col[np.abs(col) > 3]

798   -3.548824
Name: 2, dtype: float64

In [48]:
# Чтобы выбрать все строки, в которых встречаются значения, по абсолютной
# величине превышающие 3, мы можем воспользоваться методом any для
# булева объекта DataFrame:

data[(np.abs(data) > 3).any(1)]

,0,1,2,3
88,3.927528,-0.255126,0.854137,-0.364807
92,0.194788,-0.655054,-0.565230,3.176873
296,-3.399312,-0.974657,-0.685312,-0.645858
314,-0.817649,0.050188,1.951312,3.260383
391,-3.745356,-1.520113,-0.346839,-0.696918
489,-1.341493,-0.293333,-0.242459,-3.056990
513,0.425384,-3.428254,-0.296336,-0.439938
576,-0.085420,0.275144,1.179227,-3.184377
798,-0.150923,-0.362528,-3.548824,1.553205
890,1.397822,3.366626,-2.372214,0.851010


In [49]:
# Можно также присваивать значения данным, удовлетворяющим этому критерию. 
# Следующий код срезает значения, выходящие за границы интервала от –3 до 3:

data[np.abs(data) > 3] = np.sign(data) * 3

# Выражение np.sign(data) равно 1 или –1 в зависимости от того, является
# значение data положительным или отрицательным:

In [50]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.001204,-0.060553,0.070680,0.018904
std,0.989376,0.992459,0.990177,1.000171
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.636641,-0.771156,-0.591841,-0.641675
50%,-0.010997,-0.115171,0.094503,-0.000015
75%,0.659019,0.624615,0.787953,0.676536
max,3.000000,3.000000,2.653656,3.000000


## Перестановки и случайная выборка

Переставить (случайным образом переупорядочить) объект Series или строки объекта DataFrame легко с помощью функции numpy.random.permutation. Если
передать функции permutation длину оси, для которой производится перестановка, то будет возвращен массив целых чисел, описывающий новый порядок:

In [51]:
df = pd.DataFrame(np.arange(5 * 4).reshape(5, 4))

df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [52]:
sampler = np.random.permutation(5, )

sampler

array([2, 1, 4, 3, 0])

In [53]:
# массив можно использовать для индексирования на основе iloc
# или, что эквивалентно, передать функции take:

df.take([2, 4, 0, 3, 1])
# или df.take(sampler)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15
1,4,5,6,7


Чтобы выбрать случайное подмножество без возвращения, можно использовать метод sample объектов Series и DataFrame:

In [54]:
df.sample(n=3)

,0,1,2,3
4,16,17,18,19
1,4,5,6,7
2,8,9,10,11


Чтобы сгенерировать выборку с возвращением (когда разрешается выбирать один и тот же элемент несколько раз), передайте методу sample аргумент
replace=True:

In [55]:
df.sample(n=10, replace = True)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
2,8,9,10,11
4,16,17,18,19
3,12,13,14,15
2,8,9,10,11


## Вычисление индикаторных переменных

Еще одно преобразование, часто встречающееся в статистическом моделировании и машинном обучении, – преобразование категориальной переменной
в фиктивную, или индикаторную, матрицу. Если в столбце объекта DataFrame
встречается k различных значений, то можно построить матрицу или объект
DataFrame с k столбцами, содержащими только нули и единицы. В библиотеке
pandas для этого имеется функция get_dummies, хотя нетрудно написать и свою
собственную. Вернемся к приведенному выше примеру DataFrame:

In [56]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [57]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


Иногда желательно добавить префикс к столбцам индикаторного объекта
DataFrame, который затем можно будет слить с другими данными. У функции
get_dummies для этой цели предусмотрен аргумент prefix:

In [58]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [59]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


Если некоторая строка DataFrame принадлежит нескольким категориям,
то ситуация немного усложняется. Рассмотрим набор данных MovieLens 1M,
который будет более подробно исследован в главе 14:

In [60]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

D:\ANACONDA\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Чтобы добавить индикаторные переменные для каждого жанра, данные
придется немного переформатировать. Сначала построим список уникальных жанров, встречающихся в наборе данных:

In [61]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
    
genres = pd.unique(all_genres)

genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

Для построения индикаторного DataFrame можно, например, начать с объекта DataFrame, содержащего только нули:

In [62]:
zero_matrix = np.zeros((len(movies), len(genres)))
print(zero_matrix.shape)
zero_matrix

(3883, 18)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [63]:
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Затем перебираем все фильмы и присваиваем элементам в каждой строке
объекта dummies значение 1. Для этого воспользуемся атрибутом dummies.columns, чтобы вычислить индексы столбцов для каждого жанра:

In [64]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [65]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

Далее можно использовать .iloc, чтобы установить значения для этих индексов:

In [66]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [67]:
# После этого можно, как и раньше, соединить с movies:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                             1.0
Genre_Children's                            1.0
                               ...             
Genre_War                                   0.0
Genre_Musical                               0.0
Genre_Mystery                               0.0
Genre_Film-Noir                             0.0
Genre_Western                               0.0
Name: 0, Length: 21, dtype: object

В статистических приложениях бывает полезно сочетать функцию get_dummies с той или иной функцией дискретизации, например cut:

In [68]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

In [69]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [70]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


# 7.3. Манипуляции со строками

В более сложных ситуациях,
когда нужно сопоставлятьь текст с образцами, на помощь приходят регулярные выражения.

## Методы строковых объектов

##### Таблица 7.3. Встроенные в Python методы строковых объектов
- count - Возвращает количество неперекрывающихся вхождений подстроки в строку
- endswith, startswith - Возвращает True, если строка оканчивается (начинается) указанной подстрокой
- join - Использовать данную строку как разделитель при конкатенации
последовательности других строк
- index - Возвращает позицию первого символа подстроки в строке. Если подстрока
не найдена, возбуждает исключение ValueError
- find - Возвращает позицию первого символа первого вхождения подстроки в строку,
как и index. Но если строка не найдена, то возвращает –1
- rfind Возвращает позицию первого символа последнего вхождения подстроки
в строку. Если строка не найдена, то возвращает –1
- replace - Заменяет вхождения одной строки другой строкой
- strip, rstrip,
lstrip - 
Удаляет пробельные символы, в том числе символы новой строки в начале
и (или) конце строки
- split - Разбивает строку на список подстрок по указанному разделителю
- lower - Преобразует буквы в нижний регистр
- upper - Преобразует буквы в верхний регистр
- ljust, rjust - Выравнивает строку по левой или правой границе соответственно. Противоположный конец строки заполняется пробелами (или каким-либо другим символом), так чтобы получилась строка как минимум заданной длины

## Регулярные выражения

Функции из модуля re можно отнести к трем категориям: сопоставление с образцом, замена и разбиение. Естественно, все они взаимосвязаны;
регулярное выражение описывает образец, который нужно найти в тексте,
а затем его уже можно применять для разных целей. 

In [71]:
import re

text = "foo bar\t baz \tqux"

re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

При обращении re.split('\s+', text) сначала компилируется регулярное выражение, а затем его методу split передается заданный текст. Можно просто откомпилировать регулярное выражение методом re.compile, создав тем
самым объект, допускающий повторное использование:

In [72]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

Чтобы получить список всех подстрок, отвечающих данному регулярному
выражению, следует воспользоваться методом findall:

In [73]:
regex.findall(text)

[' ', '\t ', ' \t']

С findall тесно связаны методы match и search. Если findall возвращает все
найденные в строке соответствия, то search – лишь первое. А метод match находит только соответствие, начинающееся в начале строки. В качестве не
столь тривиального примера рассмотрим блок текста и регулярное выражение, распознающее большинство адресов электронной почты:

In [74]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# Флаг re.IGNORECASE делает регулярное выражение нечувствительным к регистру
regex = re.compile(pattern, flags=re.IGNORECASE)

In [75]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

Метод search возвращает специальный объект соответствия для первого
встретившегося в тексте адреса. В нашем случае этот объект может сказать
только о начальной и конечной позициях найденного в строке образца:

In [76]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [77]:
text[m.start():m.end()]

'dave@google.com'

Метод regex.match возвращает None, потому что он находит соответствие образцу только в начале строки:

In [78]:
print(regex.match(text))

None


Метод sub возвращает новую строку, в которой вхождения образца заменены указанной строкой:

In [79]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



Предположим, что мы хотим найти почтовые адреса и в то же время разбить каждый адрес на три компонента: имя пользователя, имя домена и суффикс домена. Для этого заключим соответствующие части образца в скобки:

In [80]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

regex = re.compile(pattern, flags=re.IGNORECASE)

Метод groups объекта соответствия, порожденного таким модифицированным регулярным выражением, возвращает кортеж компонентов образца:

In [81]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

Если в образце есть группы, то метод findall возвращает список кортежей:

In [82]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

Метод sub тоже имеет доступ к группам в каждом найденном соответствии
с помощью специальных конструкций \1, \2 и т. д.:

In [83]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



##### Таблица 7.4. Методы регулярных выражений

- findall - Возвращает список всех непересекающихся образцов, найденных в строке
- finditer - Аналогичен findall, но возвращает итератор
- match - Ищет соответствие образцу в начале строки и факультативно выделяет в образце группы.
Если образец найден, возвращает объект соответствия, иначе None
- search - Ищет в строке образец; если найден, возвращает объект соответствия. В отличие от match,
образец может находиться в любом месте строки, а не только в начале
- split - Разбивает строку на части в местах вхождения образца
- sub, subn - Заменяет все (sub) или только первые n (subn) вхождений образца указанной строкой.
Чтобы в указанной строке сослаться на группы, выделенные в образце, используйте
конструкции \1, \2, ...

## Векторные строковые функции в pandas

Очистка замусоренного набора данных для последующего анализа подразумевает значительный объем манипуляций со строками и использование
регулярных выражений. В столбцах, содержащих строки, иногда встречаются отсутствующие значения:

In [84]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [85]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

Методы строк и регулярных выражений можно применить к каждому значению с помощью метода data.map (которому передается лямбда или другая
функция), но для отсутствующих значений они «грохнутся». Чтобы справиться с этой проблемой, в классе Series есть методы для операций со строками,
которые пропускают отсутствующие значения. Доступ к ним производится
через атрибут str; например, вот как можно было бы с помощью метода str.
contains проверить, содержит ли каждый почтовый адрес подстроку 'gmail':

In [86]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Регулярные выражения тоже можно так использовать, равно как и их флаги
типа IGNORECASE:

In [87]:
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [88]:
# Обращение к строкам
# matches.str.get(1)
# matches.str[0]

Вырезание строк:

In [89]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

##### Таблица 7.5. Неполный перечень векторных методов строковых объектов

- cat - Поэлементно конкатенирует строки с необязательным разделителем
- contains - Возвращает булев массив, показывающий, содержит ли каждая строка указанный образец
- count - Подсчитывает количество вхождений образца
- extract - Использует регулярное выражение с группами, чтобы выделить одну или несколько
строк из объекта Series, содержащего строки; результатом является DataFrame,
содержащий по одному столбцу на каждую группу
- endswith - Эквивалентно x.endswith(pattern) для каждого элемента
- startswith - Эквивалентно x.startswith(pattern) для каждого элемента
- findall - Возвращает список всех вхождений образца для каждой строки
- get - Доступ по индексу ко всем элементам (выбрать i-й элемент)
- isalnum - Эквивалентно встроенному методу str.isalnum
- isalpha - Эквивалентно встроенному методу str.isalpha
- isdecimal - Эквивалентно встроенному методу str.isdecimal
- isdigit - Эквивалентно встроенному методу str.isdigit
- islower - Эквивалентно встроенному методу str.islower
- isnumeric - Эквивалентно встроенному методу str.isnumeric
- isupper - Эквивалентно встроенному методу str.isupper
- join - Объединяет строки в каждом элементе Series, вставляя между ними указанный разделитель
- len - Вычисляет длину каждой строки
- lower, upper - Преобразование регистра; эквивалентно x.lower() или x.upper() для каждого элемента
- match - Вызывает re.match с указанным регулярным выражением для каждого элемента,
возвращает список выделенных групп
- pad - Дополняет строки пробелами слева, справа или с обеих сторон
- center - Эквивалентно pad(side='both')
- repeat - Дублирует значения; например, s.str.repeat(3) эквивалентно x * 3 для каждой строки
- replace - Заменяет вхождения образца указанной строкой
- slice - Вырезает каждую строку в объекте Series
- split - Разбивает строки по разделителю или по регулярному выражению
- strip - Убирает пробельные символы, в том числе знак новой строки, с обеих сторон строки
- rstrip - Убирает пробельные символы справа
- lstrip - Убирает пробельные символы слева